##### Imports

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

# Text Extraction

In [2]:
import fitz 
textbook_path = "textbook_v2.pdf"
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page in document:
        text += page.get_text()
    document.close()
    return text

# If textbook_as_text.txt exists, load it, otherwise extract text from the pdf
try:
    with open('textbook_as_text.txt', 'r', encoding="utf-8") as file:
        pdf_text = file.read()
except FileNotFoundError:
    pdf_text = extract_text_from_pdf(textbook_path)

    with open('textbook_as_text.txt', 'w', encoding="utf-8") as file:
        file.write(pdf_text)

## Chunking

In [3]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_text_splitters  import RecursiveCharacterTextSplitter

# Assume `textbook_content` is your raw cleaned text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
documents = text_splitter.create_documents([pdf_text])


## Vector Embedding

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings(api_key='your_openai_api_key')
vector_store = FAISS()

# Embed and index documents
for doc in documents:
    embedding = embeddings.embed_text(doc.content)
    vector_store.add_document(doc.id, embedding)


TypeError: FAISS.__init__() missing 4 required positional arguments: 'embedding_function', 'index', 'docstore', and 'index_to_docstore_id'